In [1]:
pip install fuzzywuzzy

In [2]:
pip install tqdm

In [1]:
import pandas as pd
import numpy as np
import random
from fuzzywuzzy import fuzz
from tqdm import tqdm
from sklearn.utils import shuffle

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# negative sample from different product
def generate_sample_different_product(question, filename, data):
    C = data[data['filename'] == filename][data['Possible Questions'] == question]
    C = C.reset_index(drop = True)
    D = []
    product_name = C['Product name'][0]
    country = C['country'][0]
    A = data[data['Product name'] != product_name]
    B = A['Atom'].unique()
    for b in B:
        if fuzz.ratio(b, C['Atom'][0])>80:
            B = np.setdiff1d(B, b)
    c = len(B)
    if c > 3:
        randomlist = random.sample(range(0,c), 5)
        D.append(B[randomlist[0]])
        D.append(B[randomlist[1]])
        D.append(B[randomlist[2]])
        D.append(B[randomlist[3]])
        D.append(B[randomlist[4]])
        return D

In [3]:
# negative samples from same product and different country
def generate_sample_same_product_different_country(question, filename, data):
    C = data[data['filename'] == filename][data['Possible Questions'] == question]
    C = C.reset_index(drop = True)
    D = []
    product_name = C['Product name'][0]
    country = C['country'][0]
    A = data[data['Product name'] == product_name][data['country'] != country]
    B = A['Atom'].unique()
    for b in B:
        if fuzz.ratio(b, C['Atom'][0])>80:
            B = np.setdiff1d(B, b)
    c = len(B)
    if c > 3:
        randomlist = random.sample(range(0,c), 3)
        D.append(B[randomlist[0]])
        D.append(B[randomlist[1]])
        D.append(B[randomlist[2]])
        return D
    elif c == 3:
        D.append(B[0])
        D.append(B[1])
        D.append(B[2])
        return D
    elif c==2:
        D.append(B[0])
        D.append(B[1])
        return D
    else:
        return D

In [4]:
# negative samples from same product and same country
def generate_sample_same_product_same_country(question, filename, data):
    C = data[data['filename'] == filename][data['Possible Questions'] == question] #generate -ve atoms for this question
    C = C.reset_index(drop = True)
    D = [] #store -ve samples
    E = [] #
    F = [] #
    product_name = C['Product name'][0] #product_name
    country = C['country'][0] #country_name
    A = data[data['Product name'] == product_name][data['country'] == country] #filter df for same product and same country
    B = A['Atom'].unique() #unique atoms to be considered for generating -ve samples
    for b in B:
        if fuzz.ratio(b, C['Atom'][0])>80: #if unique atom and true atom is similar
            B = np.setdiff1d(B, b) # ignore true atom from unique atom
    c = len(B)
    if c > 2:
        randomlist = random.sample(range(0,c), 2) # indexes of -ve atoms
        D.append(B[randomlist[0]])
        D.append(B[randomlist[1]])
        return D
    elif c == 2:
        D.append(B[0])
        D.append(B[1])
        return D
    else:
        return D

In [5]:
Answers = pd.read_excel('/content/KnowledgeBase_Phase_2_Clubbed_ATOMs_6_July.xlsx')

In [6]:
product = []
country = []
Atom = []
question = []
filename = []
def split_question(a_string):
  x = a_string.split('\n')
  a = []
  for i in range(len(x)):
    if x[i] !='':
      a.append(x[i])
  return a
for i in range(Answers.shape[0]):
  x = split_question(Answers['Possible Questions'][i])
  for j in range(len(x)):
    filename.append(Answers['filename'][i])
    product.append(Answers['Product name'][i])
    country.append(Answers['country'][i])
    Atom.append(Answers['para_text'][i])
    question.append(x[j])

In [7]:
dataset = pd.DataFrame()
dataset['filename'] = filename
dataset['Product name'] = product
dataset['country'] = country
dataset['Atom'] = Atom
dataset['Possible Questions'] = question

In [8]:
Answers = dataset

In [9]:
Answers.shape

(11449, 5)

In [10]:
Answers = Answers.dropna()

In [12]:
Answers = Answers.reset_index(drop=True)

In [13]:
a = []
b = []
c = []
d = []
e = []

In [ ]:
for i in tqdm(range(Answers.shape[0])):
  for d1 in generate_sample_same_product_same_country(Answers['Possible Questions'][i], Answers['filename'][i], Answers):
    a.append(d1)
    b.append(Answers['Possible Questions'][i]) 
    c.append(Answers['Product name'][i])
    d.append(Answers['country'][i])
    e.append(Answers['Atom'][i]) 

In [ ]:
a2 = []
b2 = []
c2 = []
d2 = []
e2 = []

In [ ]:
for i in tqdm(range(Answers.shape[0])):
  for d1 in generate_sample_same_product_different_country(Answers['Possible Questions'][i], Answers['filename'][i], Answers):
    a2.append(d1)
    b2.append(Answers['Possible Questions'][i]) 
    c2.append(Answers['Product name'][i])
    d2.append(Answers['country'][i])
    e2.append(Answers['Atom'][i])

In [ ]:
for i in tqdm(range(Answers.shape[0])):
  for d1 in generate_sample_different_product(Answers['Possible Questions'][i], Answers['filename'][i], Answers):
    a3.append(d1)
    b3.append(Answers['Possible Questions'][i]) 
    c3.append(Answers['Product name'][i])
    d3.append(Answers['country'][i])
    e3.append(Answers['Atom'][i])

In [ ]:
len(Answer1),len(Question1),len(product1),len(country1),len(True_atom1)

In [ ]:
len(Answer2),len(Question2),len(product2),len(country2),len(True_atom2)

In [ ]:
len(Answer3),len(Question3),len(product3),len(country3),len(True_atom3)

In [ ]:
Answer = a+a2+a3
Question = b+b2+b3
product = c+c2+c3
country = d+d2+d3
True_atom = e+ e2+ e3